Thực hiện nối bác bảng dữ liệu để tổng hợp dữ liệu cần thiết

In [ ]:
import pandas as pd
df_sv = pd.read_csv('student_info_clean.csv')
df_hp = pd.read_csv('tuition_fees_clean.csv')
df_dong = pd.read_csv('payment_records_clean.csv')
# Gộp bảng
df_final = pd.merge(df_hp, df_sv, on='student_id', how='left')
df_final = pd.merge(df_final, df_dong_sum, on=['student_id', 'semester'], how='left')
df_final['tong_dong'] = df_final['tong_dong'].fillna(0)
df_final['so_lan_tre'] = df_final['so_lan_tre'].fillna(0)
df_final['con_no'] = df_final['tuition_amount'] - df_final['tong_dong']


vde 1,2:Tính tổng học phí phải nộp/Tính tổng số tiền đã thanh toán

In [30]:
tong_hoc_phi = df_final['tuition_amount'].sum()
tong_da_thanh_toan = df_final['tong_dong'].sum()
print(f" TỔNG HỌC PHÍ PHẢI NOP:      {tong_hoc_phi:>15} VNĐ")
print(f" TỔNG SỐ TIỀN ĐÃ THANH TOÁN: {tong_da_thanh_toan:>15} VNĐ")

 TỔNG HỌC PHÍ PHẢI NOP:          755000000.0 VNĐ
 TỔNG SỐ TIỀN ĐÃ THANH TOÁN:     189000000.0 VNĐ


vde 3: Tính số lượng sinh viên thanh toán trễ

In [36]:
sv_dong_tre = df_final[df_final['so_lan_tre'] > 0]['student_id'].nunique()
print(f" SỐ LƯỢNG SINH VIÊN ĐÓNG TRỄ: {sv_dong_tre:>14} sinh viên")

 SỐ LƯỢNG SINH VIÊN ĐÓNG TRỄ:              2 sinh viên


vde4:Tính tỷ lệ sinh viên thanh toán một phần

In [32]:
sv_chua_het = df_final[(df_final['tong_dong'] > 0) & (df_final['con_no'] > 0)]
ty_le_chua_het = (len(sv_chua_het) / len(df_final)) * 100
print(f" TỶ LỆ SV ĐÓNG CHƯA HẾT:      {ty_le_chua_het:>14.2f} %")

 TỶ LỆ SV ĐÓNG CHƯA HẾT:                5.83 %


vde5: so sánh đỗ trễ hạn học phí giữa các loại hình đào tạo

In [33]:
print(" SO SÁNH MỨC ĐỘ TUÂN THỦ THỜI HẠN GIỮA CÁC HỆ ĐÀO TẠO:")
so_sanh = df_final.groupby('program_type').agg(
    tong_sv=('student_id', 'count'),
    so_sv_tre=('so_lan_tre', lambda x: (x > 0).sum())
).reset_index()

so_sanh['ty_le_tre'] = (so_sanh['so_sv_tre'] / so_sanh['tong_sv'] * 100).round(2)
so_sanh = so_sanh.sort_values(by='ty_le_tre', ascending=True) # Hệ nào ít trễ nhất đứng đầu

print(so_sanh[['program_type', 'tong_sv', 'so_sv_tre', 'ty_le_tre']].to_string(index=False, 
      header=['Hệ Đào Tạo', 'Tổng SV', 'Số SV Đóng Trễ', 'Tỷ Lệ Trễ (%)']))



 SO SÁNH MỨC ĐỘ TUÂN THỦ THỜI HẠN GIỮA CÁC HỆ ĐÀO TẠO:
      Hệ Đào Tạo Tổng SV Số SV Đóng Trễ Tỷ Lệ Trễ (%)
           c l c      11              0             0
      chinh  quy      11              0             0
       chinh quy       3              0             0
       chính quy       9              0             0
             clc       9              0             0
      lien thong       1              0             0
      liên thông       3              0             0
vua lam  vua hoc      11              0             0
 vua lam vua hoc       4              0             0
 vừa làm vừa học       9              1            11
